Group Name: AG 10.

Student Name (Student ID):

1. Ye Fangda (A0310135Y)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

4. xxxx xxxxx (xxxxxxx)

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](Maze.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search

    f. Greedy Best-first Search

2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the form of a table.

3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.


Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.).

Note 2: The code should be written in a single jupyter notebook file.

In [1]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)

    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [2]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost


In [3]:
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.

def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

In [4]:
from collections import deque
import heapq

FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)

    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]

    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

In [5]:
MAZE_ROWS = 16
MAZE_COLUMNS = 24
ACTIONS = {'UP', 'LEFT', 'RIGHT', 'DOWN'}
SHADE = {(6, 9), (7, 9), (8, 9), (9, 9), (10, 9), (14, 9), (10, 10), (11, 10), (12, 10), (13, 10), (10, 12), (11, 12), (10, 13), (11, 13)}
# SHADE = {(1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9), (10, 9), (12, 9), (10, 10), (11, 10), (12, 10), (10, 12), (11, 12), (10, 13), (11, 13)}

# MAP = [[0 for _ in range(MAZE_COLUMNS)] for _ in range(MAZE_ROWS)]
# for i in range(MAZE_ROWS):
#     for j in range(MAZE_COLUMNS):
#         if (i, j) in SHADE:
#             MAP[i][j] = 1

class Maze(Problem):

    def __init__(self, initial=None, goal=None):
        Problem.__init__(self, initial=initial, goal=goal)

    def h1(self, node):
        return abs(node.state[0] - self.goal[0]) + abs(node.state[1] - self.goal[1]) * 10

    def h2(self, node):
        return abs(node.state[0] - self.goal[0]) + abs(node.state[1] - self.goal[1])

    def action_cost(self, s, a, s1):
        return 10 if a in {'LEFT', 'RIGHT'} else 1

    def result(self, state, action):
        x, y = state
        if action == 'UP':
            x += 1
        elif action == 'DOWN':
            x -= 1
        elif action == 'LEFT':
            y -= 1
        elif action == 'RIGHT':
            y += 1
        return (x, y)

    def actions(self, state):
        allowed_actions = ACTIONS.copy()
        if state[0] == 0:
            allowed_actions.discard('DOWN')
        if state[0] == MAZE_ROWS - 1:
            allowed_actions.discard('UP')
        if state[1] == 0:
            allowed_actions.discard('LEFT')
        if state[1] == MAZE_COLUMNS - 1:
            allowed_actions.discard('RIGHT')
        if (state[0] + 1, state[1]) in SHADE:
            allowed_actions.discard('UP')
        if (state[0] - 1, state[1]) in SHADE:
            allowed_actions.discard('DOWN')
        if (state[0], state[1] + 1) in SHADE:
            allowed_actions.discard('RIGHT')
        if (state[0], state[1] - 1) in SHADE:
            allowed_actions.discard('LEFT')
        return allowed_actions


### Testing the model

In [6]:
#Testing Maze class
#Note that we perform additional tests to validate the modeling

#An instance of Maze problem; initialize the Maze class with appropriate values
m1 = Maze(initial = (8, 10), goal = (11, 9))

# Testing the methods in Maze class
assert m1.result((4,12),'UP') == (5,12)
assert m1.action_cost((4,12),'UP',(5,12)) == 1
assert m1.actions((4,12)) == {'UP', 'LEFT', 'RIGHT', 'DOWN'}

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search

    f. Greedy Best-first Search

In [7]:
#Your code for breadth-first search
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

result_bfs = breadth_first_search(m1)
for s in path_states(result_bfs):
    print(s)
print(result_bfs.path_cost)
# 不是最优解，因为不是cost_unit

(8, 10)
(7, 10)
(6, 10)
(5, 10)
(5, 9)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
(11, 8)
(11, 9)
39


In [8]:
#Your code for  depth-first search with cycle-check
import math

def depth_first_search(problem):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    reached = set()
    while frontier:
        node = frontier.pop()
        print(node)
        if problem.is_goal(node.state):
            return node
        reached.add(node.state)
        for child in expand(problem, node):
            if child.state not in reached:
                print('state:', child.state)
                frontier.append(child)
    return result

result_dfs = depth_first_search(m1)
for s in path_states(result_dfs):
    print(s)
print(result_dfs.path_cost)
# Right Down
# Up Down Right
# 不同action的优先程度能够很大的影响结果

<(8, 10)>
state: (8, 11)
state: (9, 10)
state: (7, 10)
<(7, 10)>
state: (7, 11)
state: (6, 10)
<(6, 10)>
state: (6, 11)
state: (5, 10)
<(5, 10)>
state: (5, 11)
state: (5, 9)
state: (4, 10)
<(4, 10)>
state: (4, 11)
state: (4, 9)
state: (3, 10)
<(3, 10)>
state: (3, 11)
state: (3, 9)
state: (2, 10)
<(2, 10)>
state: (2, 11)
state: (2, 9)
state: (1, 10)
<(1, 10)>
state: (1, 11)
state: (1, 9)
state: (0, 10)
<(0, 10)>
state: (0, 11)
state: (0, 9)
<(0, 9)>
state: (1, 9)
state: (0, 8)
<(0, 8)>
state: (1, 8)
state: (0, 7)
<(0, 7)>
state: (1, 7)
state: (0, 6)
<(0, 6)>
state: (1, 6)
state: (0, 5)
<(0, 5)>
state: (1, 5)
state: (0, 4)
<(0, 4)>
state: (1, 4)
state: (0, 3)
<(0, 3)>
state: (1, 3)
state: (0, 2)
<(0, 2)>
state: (1, 2)
state: (0, 1)
<(0, 1)>
state: (1, 1)
state: (0, 0)
<(0, 0)>
state: (1, 0)
<(1, 0)>
state: (1, 1)
state: (2, 0)
<(2, 0)>
state: (2, 1)
state: (3, 0)
<(3, 0)>
state: (3, 1)
state: (4, 0)
<(4, 0)>
state: (4, 1)
state: (5, 0)
<(5, 0)>
state: (5, 1)
state: (6, 0)
<(6, 0)>
state:

In [9]:
#Your code for  iterative Deepening search with cycle-check
import sys
def iterative_deepening_search(problem):
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result

def depth_limited_search(problem, limit=100):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) > limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result

def is_cycle(node, k=30):
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

result_ids = iterative_deepening_search(m1)
for s in path_states(result_ids):
    print(s)
print(result_ids.path_cost)

(8, 10)
(7, 10)
(6, 10)
(5, 10)
(5, 9)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
(11, 8)
(11, 9)
39


In [10]:
#Your code for uniform-cost search
def uniform_cost_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

def g(n):
    return n.path_cost

result_ucs = uniform_cost_search(m1, f=g)
for s in path_states(result_ucs):
    print(s)
print(result_ucs.path_cost)

(8, 10)
(7, 10)
(6, 10)
(5, 10)
(5, 9)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
(11, 8)
(11, 9)
39


In [11]:
#Your code for A* Search
def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return uniform_cost_search(problem, f=lambda n: g(n) + h(n))

def g(n):
    return n.path_cost

result_a_star = astar_search(m1, h=m1.h1)
for s in path_states(result_a_star):
    print(s)
print('h1:', result_a_star.path_cost)

result_a_star = astar_search(m1, h=m1.h2)
for s in path_states(result_a_star):
    print(s)
print('h2:', result_a_star.path_cost)



(8, 10)
(7, 10)
(6, 10)
(5, 10)
(5, 9)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
(11, 8)
(11, 9)
h1: 39
(8, 10)
(7, 10)
(6, 10)
(5, 10)
(5, 9)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
(11, 8)
(11, 9)
h2: 39


In [12]:
#Your code for greedy-best first search
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return uniform_cost_search(problem, f=h)

result_greedy = greedy_bfs(m1, h=m1.h1)
for s in path_states(result_greedy):
    print(s)
print('h1:', result_greedy.path_cost)

result_greedy = greedy_bfs(m1, h=m1.h2)
for s in path_states(result_greedy):
    print(s)
print('h2:', result_greedy.path_cost)

(8, 10)
(7, 10)
(6, 10)
(5, 10)
(5, 9)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
(11, 8)
(11, 9)
h1: 39
(8, 10)
(9, 10)
(9, 11)
(10, 11)
(11, 11)
(12, 11)
(13, 11)
(14, 11)
(14, 10)
(15, 10)
(15, 9)
(15, 8)
(14, 8)
(13, 8)
(12, 8)
(11, 8)
(11, 9)
h2: 61


2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the following table.


 Algorithm             |Number of expanded nodes  | Maximum Frontier Size  |  Path Cost                  |
|:---------------------|:-------------------------|:-----------------------|:----------
| Breadth-First Search |                          |                        |
| DFS with cycle check |                          |                        |
| IDS with cycle check |                          |                        |
| UCS                  |                          |                        |
| A* Search            |                          |                        |
| GBFS                 |                          |                        |

3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.

